In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

BASE_PATH = r"C:\Users\Impana\Downloads\invoice-classification\\"

train_df = pd.read_csv(os.path.join(BASE_PATH, "data", "sroie", "D3_sroie_WNtrain10k.csv"))
test_df  = pd.read_csv(os.path.join(BASE_PATH, "data", "sroie", "D2_sroie_test.csv"))

print("Train:", train_df.shape, "Test:", test_df.shape)
print(train_df['category'].value_counts())
print(test_df['category'].value_counts())

Train: (10000, 2) Test: (68, 7)
category
OTHER          6497
RETAIL         1033
FOOD            936
STATIONERY      670
HARDWARE        661
ELECTRONICS      84
SERVICES         76
HOTEL            43
Name: count, dtype: int64
category
OTHER         45
FOOD           7
RETAIL         7
HARDWARE       5
STATIONERY     4
Name: count, dtype: int64


In [7]:
X_train_text = train_df['text'].astype(str).tolist()
X_test_text  = test_df['text_clean'].astype(str).tolist()

le = LabelEncoder()
y_train = le.fit_transform(train_df['category'].astype(str).values)
y_test  = le.transform(test_df['category'].astype(str).values)
num_classes = len(le.classes_)
print("Classes:", le.classes_)

Classes: ['ELECTRONICS' 'FOOD' 'HARDWARE' 'HOTEL' 'OTHER' 'RETAIL' 'SERVICES'
 'STATIONERY']


In [8]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)
class_weights_arr = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weights = {int(c): float(w) for c, w in zip(classes, class_weights_arr)}
print("Class weights:", class_weights, " -> ", le.classes_)

Class weights: {0: 14.880952380952381, 1: 1.3354700854700854, 2: 1.8910741301059002, 3: 29.069767441860463, 4: 0.19239649068800985, 5: 1.2100677637947725, 6: 16.44736842105263, 7: 1.8656716417910448}  ->  ['ELECTRONICS' 'FOOD' 'HARDWARE' 'HOTEL' 'OTHER' 'RETAIL' 'SERVICES'
 'STATIONERY']


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

embedding_dim = 48  # smaller
lstm_units = 32     # smaller than 64

model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              input_length=max_len),
    Bidirectional(LSTM(lstm_units, dropout=0.4, recurrent_dropout=0.4)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dropout(0.4),
    Dense(num_classes, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-4),
    metrics=['accuracy']
)

model.summary()


c:\Users\Impana\anaconda3\envs\invoice\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
batch_size = 64
epochs = 12   # a bit more, but with class weights and stronger dropout

history = model.fit(
    X_train_seq, y_train,
    validation_split=0.1,
    epochs=epochs,
    batch_size=batch_size,
    class_weight=class_weights,
    verbose=1
)


Epoch 1/12
141/141 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.1220 - loss: 2.0628 - val_accuracy: 0.7490 - val_loss: 2.0248
Epoch 2/12
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.2803 - loss: 2.0211 - val_accuracy: 0.4180 - val_loss: 1.9439
Epoch 3/12
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.3076 - loss: 1.8136 - val_accuracy: 0.5970 - val_loss: 1.5546
Epoch 4/12
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.3989 - loss: 1.4570 - val_accuracy: 0.6810 - val_loss: 1.2089
Epoch 5/12
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.4857 - loss: 1.1673 - val_accuracy: 0.7000 - val_loss: 1.0116
Epoch 6/12
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.5591 - loss: 0.9610 - val_accuracy: 0.7340 - val_loss: 0.8574
Epoch 7/12
141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - accuracy: 0.6111 - loss: 0.7961 - val_accuracy: 0.7700 - val_loss: 0.7202
Epoch 8/12
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - accuracy: 0.6520 - loss: 0.6638 - val_acc

In [12]:
from sklearn.metrics import classification_report
import numpy as np

labels_present = np.unique(y_test)

print("Labels present in test:", labels_present)
print("Corresponding class names:", le.classes_[labels_present])

print(classification_report(
    y_test,
    y_pred,
    labels=labels_present,
    target_names=le.classes_[labels_present]
))


Labels present in test: [1 2 4 5 7]
Corresponding class names: ['FOOD' 'HARDWARE' 'OTHER' 'RETAIL' 'STATIONERY']
              precision    recall  f1-score   support

        FOOD       0.56      0.71      0.62         7
    HARDWARE       0.56      1.00      0.71         5
       OTHER       0.90      0.60      0.72        45
      RETAIL       0.35      0.86      0.50         7
  STATIONERY       0.50      0.25      0.33         4

   micro avg       0.66      0.65      0.65        68
   macro avg       0.57      0.68      0.58        68
weighted avg       0.76      0.65      0.66        68



In [3]:
max_words = 10000
max_len = 40  # addresses are short

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train_text),
                            maxlen=max_len, padding='post')
X_test_seq  = pad_sequences(tokenizer.texts_to_sequences(X_test_text),
                            maxlen=max_len, padding='post')

vocab_size = min(max_words, len(tokenizer.word_index) + 1)
print("Vocab size:", vocab_size)


Vocab size: 703


In [4]:
embedding_dim = 64  # smaller than invoice model; data is smaller

model = Sequential([
    Embedding(input_dim=vocab_size,
              output_dim=embedding_dim,
              input_length=max_len),
    Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    metrics=['accuracy']
)

model.summary()


c:\Users\Impana\anaconda3\envs\invoice\lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [5]:
batch_size = 64
epochs = 10

history = model.fit(
    X_train_seq, y_train,
    validation_split=0.1,
    epochs=epochs,
    batch_size=batch_size,
    verbose=1
)


Epoch 1/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 14s 57ms/step - accuracy: 0.6503 - loss: 1.2089 - val_accuracy: 0.7730 - val_loss: 0.8115
Epoch 2/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.8213 - loss: 0.6295 - val_accuracy: 0.8680 - val_loss: 0.4391
Epoch 3/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.8830 - loss: 0.4099 - val_accuracy: 0.8970 - val_loss: 0.3319
Epoch 4/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.8973 - loss: 0.3324 - val_accuracy: 0.9070 - val_loss: 0.2906
Epoch 5/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.9057 - loss: 0.2925 - val_accuracy: 0.9090 - val_loss: 0.2624
Epoch 6/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.9087 - loss: 0.2737 - val_accuracy: 0.9150 - val_loss: 0.2596
Epoch 7/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 41ms/step - accuracy: 0.9124 - loss: 0.2558 - val_accuracy: 0.9220 - val_loss: 0.2365
Epoch 8/10
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.9188 - loss: 0.2436 - val_acc

In [6]:
test_loss, test_acc = model.evaluate(X_test_seq, y_test, verbose=0)
print("Test accuracy:", test_acc)

y_pred_prob = model.predict(X_test_seq, verbose=0)
y_pred = np.argmax(y_pred_prob, axis=1)

labels_test = np.unique(y_test)
print(classification_report(
    y_test,
    y_pred,
    labels=labels_test,
    target_names=le.classes_[labels_test]
))


Test accuracy: 0.8088235259056091
              precision    recall  f1-score   support

        FOOD       1.00      0.43      0.60         7
    HARDWARE       0.67      0.40      0.50         5
       OTHER       0.80      0.98      0.88        45
      RETAIL       0.83      0.71      0.77         7
  STATIONERY       1.00      0.25      0.40         4

    accuracy                           0.81        68
   macro avg       0.86      0.55      0.63        68
weighted avg       0.83      0.81      0.78        68



#BERT

In [6]:
import os
import pandas as pd
import numpy as np

from datasets import Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

import evaluate
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)

In [7]:
BASE_PATH = r"C:\Users\Impana\Downloads\invoice-classification\\"

train_df = pd.read_csv(os.path.join(BASE_PATH, "data", "sroie", "D3_sroie_WNtrain10k.csv"))
test_df  = pd.read_csv(os.path.join(BASE_PATH, "data", "sroie", "D2_sroie_test.csv"))

print(train_df.shape, test_df.shape)
print(train_df['category'].value_counts())
print(test_df['category'].value_counts())

(10000, 2) (68, 7)
category
OTHER          6497
RETAIL         1033
FOOD            936
STATIONERY      670
HARDWARE        661
ELECTRONICS      84
SERVICES         76
HOTEL            43
Name: count, dtype: int64
category
OTHER         45
FOOD           7
RETAIL         7
HARDWARE       5
STATIONERY     4
Name: count, dtype: int64


In [8]:
le = LabelEncoder()
train_df['label'] = le.fit_transform(train_df['category'].astype(str))
test_df['label']  = le.transform(test_df['category'].astype(str))

num_labels = len(le.classes_)
print("Classes:", num_labels, le.classes_)

Classes: 8 ['ELECTRONICS' 'FOOD' 'HARDWARE' 'HOTEL' 'OTHER' 'RETAIL' 'SERVICES'
 'STATIONERY']


In [9]:
train_hf = Dataset.from_pandas(train_df[['text', 'label']])
test_hf  = Dataset.from_pandas(test_df[['text_clean', 'label']].rename(columns={'text_clean': 'text'}))


In [10]:
model_name = "google-bert/bert-base-uncased"  # or "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

max_len = 64

def preprocess(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=max_len,
    )

train_tokenized = train_hf.map(preprocess, batched=True)
test_tokenized  = test_hf.map(preprocess, batched=True)

train_tokenized = train_tokenized.remove_columns(["text"])
test_tokenized  = test_tokenized.remove_columns(["text"])

train_tokenized.set_format("torch")
test_tokenized.set_format("torch")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Impana\anaconda3\envs\invoice\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Impana\.cache\huggingface\hub\models--google-bert--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/68 [00:00<?, ? examples/s]

In [11]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
)

accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]
    f1_macro = f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"]
    f1_weighted = f1_metric.compute(predictions=preds, references=labels, average="weighted")["f1"]
    return {
        "accuracy": acc,
        "f1_macro": f1_macro,
        "f1_weighted": f1_weighted,
    }

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
batch_size = 16

training_args = TrainingArguments(
    output_dir="./sroie_bert",
    num_train_epochs=5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=2e-5,
    weight_decay=0.01,
)


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()
metrics = trainer.evaluate()
print(metrics)


In [ ]:
preds_output = trainer.predict(test_tokenized)
y_pred = np.argmax(preds_output.predictions, axis=-1)
y_true = np.array(test_tokenized["label"])

print("Test accuracy:", metrics["eval_accuracy"])
print(classification_report(
    y_true,
    y_pred,
    target_names=le.classes_
))
